# Projeto 3 - MC886 Aprendizado de Máquina

Victor Prado Palmerini - 178061

Erik Soares - 155261

## Introdução

Este é o Projeto 2 da disciplina MC886 - Aprendizado de Máquina - 1S2021

Este trabalho é dividido em 2 partes que consistem em diferentes abordagens para resolver de obter o melhor modelo do jogo PacMan para jogar sozinho e atingir a vitória com o maior score possível. Na 1ª parte o modelo é treinado utilizando **Programação Genética**. E na 2ª parte o modelo utiliza **Aprendizado por Reforço**.

O jogo utilizado é uma implementação obtida por este [link](http://cs.brynmawr.edu/Courses/cs372/fall2017/Code/search.zip)

## Descrição Geral

O projeto se baseia neste notebook e em 2 scripts em **Python 2** (`genetic.py` e `reinforcement.py`) que executam junto com o projeto do jogo e são chamados por este notebook. Além disso, há outras implementações complementares que serão citadas ao longo do relatório.

## Parte 1 - Programação Genética

Nesta parte nos preocupamos em implementar os principais conceitos relacionados à Programação Genética, como a ideia de evolução de indivíduos e gerações, crossover, mutação, entre outros. Existem vários layouts disponíveis mas neste projeto focaremos em 3: `smallClassic`, `mediumClassic` e `originalClassic`.

### Desenvolvimento

A ideia geral do nosso algoritmo de programação genética é ter a 1ª geração de PacMans com movimentos totalmente aleatórios. Os possíveis movimentos de um indivíduo são: `Stop`, `North`, `East`, `West` e `South`. Claro que isso vai depender também da posição atual do indivíduo, mas em geral esses são os movimentos válidos.

A partir da 2ª geração a ideia é que os indivíduos herdem movimentos da geração anterior e que são alterados pro crossover e mutação.

Todos os movimentos de cada indivíduo de cada geração são armazenados em arquivos que são criados em tempo de execução do jogo. Isso é útil pois assim conseguimos dar replay no jogo de qualquer indíviduo após a sua execução.

De acordo com o código do projeto base, é possível criar agents que implementam alguma funcionalidade específica para cada caso de uso. No nosso caso, queremos um agent que se movimenta aleatoriamente (1ª geração) e um agent que carregue a sua sequência de  movimentos a partir de um arquivo e siga tais movimentos no jogo (outras gerações). O agent que se movimenta aleatoriamente está implementado no arquivo `randomAgents.py` e o agent que carrega um caminho está implementado no arquivo `geneticAgents.py`. No caso do 2° agent, caso ele esgote os caminhos carregados do arquivo ele seguirá a partir daí a direção que dará a ele o maior score e que não possui um fantasma. Então ele priorizará direções válidas que contém cápsulas ou pontos; se não houver nada ao redor então ele seleciona uma direção aleatória.

O script `genetic.py` responsável pela implementação do algoritmo de programação genética possui então um loop
principal que roda o nº de gerações especificado e para depois de obter todas as gerações ou quando o melhor e o pior indivíduo de uma mesma geração tiverem variação menor que `10%` no valor do fitness (mais sobre o fitness a seguir).

Implementamos 3 métodos para o cálculo do **fitness**:

- Score médio entre os n jogos rodados pra cada indivíduo (passado como parâmetro - o default é 3)
- Nº de vitórias dos n jogos rodados pra cada indivíduo
- Score médio / nº de movimentos do caminho com melhor score

#### Função de Fitness

A função de fitness pode ser especificada por parâmetro na execução do script `genetic.py` com a flag `--fitness`. O cálculo do fitness é importante pois é ele quem mensura a performance dos indivíduos de uma mesma geração. A seguir executaremos o jogo pra cada função de fitness implementada: `score médio`, `nº de vitórias` e `score médio / nº de movimentos`.

##### - Score Médio

##### - Nº de Vitórias

##### - Score Médio / N° de Movimentos

#### Tamanho da População

Nº de indivíduos que cada geração possui. No nosso modelo estamos considerando um valor constante entre gerações.

##### - 30 Indivíduos por geração

##### - 60 Indivíduos por geração

##### - 100 Indivíduos por geração

#### Nº de jogos de cada indivíduo

O projeto base permite que a gente rode N jogos pra um mesmo indivíduo e que a gente obtenha os resultados desses jogos. Portanto esse pode ser considerado um hiperparâmetro já que a priori consegue-se uma consistência maior nos resultados quando se executa mais jogos.

##### - 1 jogo

##### - 3 jogos

##### - 5 jogos

#### Elitismo

A cada geração mantemos os melhores indivíduos inalterados pois queremos que eles continuem sendo avaliados nas próximas gerações. Aqui então variamos a porcentagem de indivíduos que são mantidos pelo menos até a próxima geração.

##### - 5% melhores

##### - 10% melhores

##### - 20% melhores

In [2]:
! python pacman.py --replay gen-0/ind-1

Replaying recorded game gen-0/ind-1.
Traceback (most recent call last):
  File "pacman.py", line 679, in <module>
    args = readCommand( sys.argv[1:] ) # Get game components based on input
  File "pacman.py", line 582, in readCommand
    replayGame(**recorded)
  File "pacman.py", line 620, in replayGame
    state = state.generateSuccessor( *action )
  File "pacman.py", line 99, in generateSuccessor
    if self.isWin() or self.isLose(): raise Exception('Can\'t generate a successor of a terminal state.')
Exception: Can't generate a successor of a terminal state.


### Análise

## Parte 2 - Aprendizado por Reforço

### Desenvolvimento

### Análise